# セグメンテーション

画像の1ピクセルごとの領域が、どんな物体に関連する領域かをラベル付けする。

## 手法
### SegNet

参考URL：http://mi.eng.cam.ac.uk/projects/segnet/

広く用いられている。
<img src="./images/seg-net.png" width="80%" style="position:relative;left:-10%;">

### U-net

参考URL：https://lmb.informatik.uni-freiburg.de/people/ronneber/u-net/

医用画像で効果を発揮。
<img src="./images/u-net.png" width="80%" style="position:relative;left:-10%;">

### 使用データ

元画像
<img src="./images/seg1.png" width="20%" style="position:relative;left:-40%;">
<br>
セグメント画像
<img src="./images/seg2.png" width="20%" style="position:relative;left:-40%;">

データ元URL：http://brainiac2.mit.edu/isbi_challenge/

### 備考：便利なUnetの学習済みモデル

https://github.com/divamgupta/image-segmentation-keras

In [1]:
import numpy as np 
import os
import skimage.io as io            # データ分析用画像処理ライブラリ（読み込み用）
import skimage.transform as trans # データ分析用画像処理ライブラリ（変換用）
from keras.models import *        # kerasのモデルモジュール全読み込み
from keras.layers import *        # kerasの層モジュール全読み込み
from keras.optimizers import *    # kerasの最適化モジュール全読み込み
from keras.preprocessing.image import ImageDataGenerator # 画像データ拡張用ライブラリ
from keras import backend as keras

Using TensorFlow backend.


In [26]:
def train_generator(batch_size):
    """学習用データ作成関数
    引数：
    　batch_size：画像枚数
    返り値：
    　処理後の元画像とマスク画像
    処理概要：
    　元画像/マスク画像の拡張後元画像を返す。
    """
    # データ拡張クラスに渡すパラメータ（データを増やしてくれる）
    data_gen_args = dict(
        rotation_range=0.2,     # 画像をランダムに回転する回転範囲
        width_shift_range=0.05, # ランダムに水平シフトする範囲
        height_shift_range=0.05,# ランダムに垂直シフトする範囲
        shear_range=0.05,       # シアー強度（反時計回りのシアー角度）
        zoom_range=0.05,        # ランダムにズームする範囲
        horizontal_flip=True,  # 水平方向に入力をランダムに反転
        fill_mode='nearest'     # 入力画像の境界周り埋め
    )
    
    # 元画像データ拡張インスタンス
    image_datagen = ImageDataGenerator(**data_gen_args)
    
    # マスク画像データ拡張インスタンス
    mask_datagen = ImageDataGenerator(**data_gen_args)
    
    # 元画像データ拡張
    image_generator = image_datagen.flow_from_directory(
        'data/membrane/train',      # ベースディレクトリのパス
        classes = ['image'],       # データ（元画像）へのパス
        class_mode = None,        # ラベルを返すか返さないか
        color_mode = "grayscale", # RGBかグレースケールか
        target_size = (256,256),  # リサイズしたい画像サイズ
        batch_size = batch_size,  # バッチサイズ
        seed = 1234               # 乱数固定：これをしないと、マスク画像と拡張処理が合わなくなる
    )
    
    # マスク画像データ拡張
    mask_generator = mask_datagen.flow_from_directory(
        'data/membrane/train',      # ベースディレクトリのパス
        classes = ['label'],       # データ（マスク画像）へのパス
        class_mode = None,        # ラベルを返すか返さないか
        color_mode = "grayscale", # RGBかグレースケールか
        target_size = (256,256),  # リサイズしたい画像サイズ
        batch_size = batch_size,  # バッチサイズ
        seed = 1234               # 乱数固定：これをしないと、元画像と拡張処理が合わなくなる
    )
    
    # 元画像は正規化
    # マスク画像はone-hotベクトル化
    for (img,mask) in zip(image_generator, mask_generator):
        # 最大値を1に
        img = img / 255
        mask = mask /255
        # one-hotベクトル化
        mask[mask > 0.5] = 1
        mask[mask <= 0.5] = 0
        # 元画像とマスク画像を返す
        yield (img,mask)

In [27]:
def unet(input_shape = (256,256,1)):
    """UNetモデルの構築
    引数：
    　input_size：入力サイズ
    返り値：
    　model：初期モデル
    処理概要：
    　Unetモデルの構築
    """
    '''シーケンシャル（連続型）
    model = Sequential()
    layers=[
        Conv2D(filters = 64, input_shape=input_shape, kernel_size = 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal'),
        Conv2D(filters = 64, kernel_size = 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal'),
        MaxPooling2D(pool_size = (2, 2)),
        Conv2D(filters = 128, kernel_size = 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal'),
        ・・・・・
    ]
    '''
    # ファンクショナル（関数型）
    # 入力層
    inputs = Input(input_shape)
    # セグメント1
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    # セグメント2
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    # セグメント3
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    # セグメント4
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
    drop4 = Dropout(0.5)(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)
    # セグメント5
    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
    drop5 = Dropout(0.5)(conv5)
    up6 = Conv2D(512, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(drop5))
    # セグメント6
    merge6 = concatenate([drop4, up6], axis = 3)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)
    up7 = Conv2D(256, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6))
    # セグメント7
    merge7 = concatenate([conv3, up7], axis = 3)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)
    up8 = Conv2D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))
    # セグメント8
    merge8 = concatenate([conv2, up8], axis = 3)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)
    up9 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8))
    # セグメント9
    merge9 = concatenate([conv1, up9], axis = 3)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge9)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv9 = Conv2D(2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    # 出力層
    conv10 = Conv2D(1, 1, activation = 'sigmoid')(conv9)
    
    # モデル定義
    model = Model(input = inputs, output = conv10)
    model.compile(
        optimizer = Adam(lr = 1e-4), 
        loss = 'binary_crossentropy', 
        metrics = ['accuracy']
    )
    
    # モデル返す
    return model

In [28]:
# モデル生成
model = unet()

# ジェネレータの定義
train_generator_img = train_generator(2)

# 訓練
model.fit_generator(
    train_generator_img,# ジェネレータ（引数は生成する画像枚数）
    steps_per_epoch=300, # 1学習当たりの学習回数
    epochs=1 # 学習回数
)

c:\users\mio\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:69: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("co...)`


Found 30 images belonging to 1 classes.
Found 30 images belonging to 1 classes.
Epoch 1/1
300/300 [==============================] - 46s 154ms/step - loss: 0.4097 - accuracy: 0.7935


In [29]:
def test_generator(batch_size):
    """テスト用データ作成関数
    引数：
    　batch_size：画像枚数
    返り値：
    　処理後の元画像
    処理概要：
    　処理後の元画像を返す
    """
    for i in range(batch_size):
        # 画像読み込み
        img = io.imread(
            os.path.join("./data/membrane/test/%d.png"%i),
            as_gray = True
        )
        
        # 最大値を1にする
        img = img / 255
        
        # リサイズ
        img = trans.resize(img,(256,256))
        img = np.reshape(img,img.shape+(1,))
        
        # グレースケール用のshapeに変換する
        img = np.reshape(img,(1,)+img.shape)
        
        # 画像を返す
        yield img

def saveResult(results):
    """予測値からマスク後画像を生成し、保存する
    引数：
    　results：ニューラルネットの予測値
    返り値：
    　
    処理概要：
    　ニューラルネットの予測値を受け取り、画像として保存する。
    """
    for i,pred in enumerate(results):
        # 予測値から、グレースケール画像の形式に変換
        img = pred[:,:,0]
        # 画像保存
        io.imsave(
            os.path.join("data/membrane/test/%d_predict.png"%i),
            img
        )

In [30]:
# ジェネレータ定義
test_generator_img = test_generator(30)

# 予測値の算出
results = model.predict_generator(
    test_generator_img, # ジェネレータ（引数は生成する画像枚数）
    steps=30, # ジェネレータが生成する画像の枚数
)

# 画像の保存
saveResult(results)

Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0

## 予測結果を見てみる

元画像
<img src="./data/membrane/test/1.png" width="20%" style="position:relative;left:-40%;">
<br>
セグメント画像
<img src="./data/membrane/test/1_predict.png" width="20%" style="position:relative;left:-40%;">